# Logistic Regression

In [1]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
from joblib import dump

In [2]:
import sys
sys.path.append('F:\\Users\\Manuel García Plaza\\Desktop\\TFG\\')

In [3]:
from notebooks.utils.classification_metrics import classification

We are going to do the same as in the breast cancer notebook

In [4]:
train =  pd.read_parquet('../../../data/model_input/train_sets/software_defect.parquet')
test =  pd.read_parquet('../../../data/model_input/validation_sets/software_defect.parquet')

In [6]:
y_train = train.defects
X_train = train.drop(columns=['defects'])

In [7]:
y_test = test.defects
X_test = test.drop(columns=['defects'])

We are using sag or saga solvers because this dataset is large

In [29]:
lr = LogisticRegression(penalty=None, solver='sag', max_iter=10000, tol=0.001)
lr_l2 = LogisticRegression(solver='sag', max_iter=10000, tol=0.001)
lr_l1 = LogisticRegression(penalty='l1', solver='saga', max_iter=1000, tol=0.001)
lr_en1 = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.25, max_iter=1000, tol=0.001)
lr_en2 = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5, max_iter=1000, tol=0.001)
lr_en3 = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.75, max_iter=1000, tol=0.001)

In [30]:
models_list = [lr, lr_l2, lr_l1, lr_en1, lr_en2 , lr_en3]
names_list = ['LR', 'LR_l2', 'LR_l1', 'LR_en_0.25', 'LR_en_0.5', 'LR_en_0.75']

In [31]:
metrics = classification(models_list, names_list, '../../../data/metrics/software_defect/logistic_regression.parquet', X_train, y_train, X_test, y_test)
metrics

,Run_Time,Train_AUC,Test_AUC,delta%
LR,32.649925,0.459964,0.468238,1.798899
LR_l2,31.034986,0.459971,0.468245,1.798859
LR_l1,43.468278,0.567207,0.578666,2.020302
LR_en_0.25,43.092880,0.567169,0.578627,2.020191
LR_en_0.5,41.569116,0.567133,0.578590,2.020216
LR_en_0.75,43.679971,0.567188,0.578649,2.020560


Every model is super bad, but we are choosing **LR_l1** as the best and we will try to improve its predictions.

In [32]:
dump(lr_l1, 'logistic_regression_best.joblib')

['logistic_regression_best.joblib']